In [1]:
import pandas as pd
import numpy as np
import random as rnd
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv('./Titanic DB/train.csv')
test = pd.read_csv('./Titanic DB/test.csv')

In [3]:
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null

In [4]:
train = train.drop(['Name', 'PassengerId', 'Ticket', 'Cabin', 'Parch', 'SibSp'], axis = 1)
test = test.drop(['Name', 'Ticket', 'Cabin', 'Parch', 'SibSp'], axis = 1)
df = [train, test]

In [5]:
mapper = {'female': 1, 'male': 0}
for data in df:
    data['Sex'] = data['Sex'].map(mapper).astype(int)

In [6]:
age_mean = train['Age'].mean()
train['Age'] = train['Age'].fillna(age_mean)
test['Age'] = test['Age'].fillna(age_mean)
df = [train, test]

In [7]:
for data in df:    
    data.loc[data['Age'] <= 16, 'Age'] = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[data['Age'] > 64, 'Age'] = 4

In [8]:
port_mode = train.Embarked.mode()[0]
for data in df:
    data['Embarked'] = data['Embarked'].fillna(port_mode)
mapper = {'S': 0, 'C': 1, 'Q': 2}
for data in df:
    data['Embarked'] = data['Embarked'].map(mapper).astype(int)

In [9]:
fare_mean = train['Fare'].mean()
test['Fare'] = test['Fare'].fillna(fare_mean)
for data in df:
    data.loc[data['Fare'] <= 7.91, 'Fare'] = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare'] = 2
    data.loc[data['Fare'] > 31, 'Fare'] = 3
    data['Fare'] = data['Fare'].astype(int)
df = [train, test]

In [10]:
X_train = train.drop("Survived", axis = 1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis = 1).copy()

In [11]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
accuracy = round(random_forest.score(X_train, Y_train) * 100, 2)
print("{}%".format(accuracy))

84.51%


In [12]:
submission = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": Y_pred})
submission.to_csv('./Titanic DB/submission.csv', index = False)